Packages:

In [ ]:
import glob
import os
import warnings


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# %pip install pydicom
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [ ]:
# Specify the file names and DataFrame variable names
file_names = ["train_df_2.csv", "test_df_2.csv"]

# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]

# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")



Restore parameters and metrtics from mlflow

In [ ]:
import mlflow
# %pip install torchsummary
from torchsummary import summary
import mlflow.pytorch

# Set the experiment name (optional, helps in identifying runs)
experiment_name = "esnet50_MRI_Classification_2"
mlflow.set_experiment(experiment_name)

# Specify the run ID of the logged model
run_id = "e2279f9e4abb43f9824a1ba402ce8168"  # Replace with your actual run ID

# Create the model URI
model_uri = f"runs:/{run_id}/model"

# Load the model
model = mlflow.pytorch.load_model(model_uri)

# Now you can use the model for inference or evaluation
model.eval()  # Set the model to evaluation mode
# Get all runs for the experiment
runs = mlflow.search_runs(experiment_names=[experiment_name])

print(runs.filter(like='params.').to_string(index=False))
# Display all metrics
metrics_columns = [col for col in runs.columns if col.startswith('metrics.')]
print(runs[metrics_columns].to_string(index=False))



In [ ]:
# Display the model summary
summary(model, input_size=(3, 224, 224))

## Explanation of Each Column of Model

### Layer (type):
This column lists the name and type of each layer in the model. For example, `Conv2d` represents a 2D convolutional layer, `BatchNorm2d` represents a 2D batch normalization layer, `ReLU` represents a rectified linear unit activation function, and so on.

### Output Shape:
This column shows the shape of the output tensor from each layer. The shape is represented as `[-1, channels, height, width]`, where `-1` indicates a variable batch size, `channels` is the number of feature maps, and `height` and `width` are the spatial dimensions of the feature maps.

### Param #:
This column indicates the number of trainable parameters in each layer. Parameters include weights and biases that are learned during training.

## Detailed Breakdown of Some Layers

### Conv2d-1:
- **Type:** Conv2d
- **Output Shape:** `[-1, 64, 112, 112]`
- **Param #:** 9,408
- **Description:** This is the first convolutional layer with 64 filters, each of size 7x7, applied to the input image. The output feature maps have a spatial resolution of 112x112.

### BatchNorm2d-2:
- **Type:** BatchNorm2d
- **Output Shape:** `[-1, 64, 112, 112]`
- **Param #:** 128
- **Description:** This layer normalizes the output of the previous convolutional layer to improve training stability and performance.

### ReLU-3:
- **Type:** ReLU
- **Output Shape:** `[-1, 64, 112, 112]`
- **Param #:** 0
- **Description:** This layer applies the ReLU activation function, which introduces non-linearity to the model.

### MaxPool2d-4:
- **Type:** MaxPool2d
- **Output Shape:** `[-1, 64, 56, 56]`
- **Param #:** 0
- **Description:** This layer performs max pooling, reducing the spatial dimensions of the feature maps to 56x56.

### Conv2d-5:
- **Type:** Conv2d
- **Output Shape:** `[-1, 64, 56, 56]`
- **Param #:** 4,096
- **Description:** This is another convolutional layer with 64 filters, each of size 3x3, applied to the feature maps from the previous layer.

### BatchNorm2d-6:
- **Type:** BatchNorm2d
- **Output Shape:** `[-1, 64, 56, 56]`
- **Param #:** 128
- **Description:** This layer normalizes the output of the previous convolutional layer.

### ReLU-7:
- **Type:** ReLU
- **Output Shape:** `[-1, 64, 56, 56]`
- **Param #:** 0
- **Description:** This layer applies the ReLU activation function.

### Bottleneck-16:
- **Type:** Bottleneck
- **Output Shape:** `[-1, 256, 56, 56]`
- **Param #:** 0
- **Description:** This is a bottleneck block, which is a key component of ResNet architectures. It consists of multiple convolutional layers and skip connections.

#### Bottleneck-172
- **Type:** Bottleneck
- **Output Shape:** `[-1, 2048, 7, 7]`
- **Param #:** 0
- **Description:** This is a bottleneck block, which is a key component of ResNet architectures. It consists of multiple convolutional layers and skip connections.

#### AdaptiveAvgPool2d-173
- **Type:** AdaptiveAvgPool2d
- **Output Shape:** `[-1, 2048, 1, 1]`
- **Param #:** 0
- **Description:** This layer performs adaptive average pooling, reducing the spatial dimensions of the feature maps to 1x1 while maintaining the number of channels.

#### Linear-174
- **Type:** Linear
- **Output Shape:** `[-1, 3]`
- **Param #:** 6,147
- **Description:** This is a fully connected (dense) layer with 3 output units, typically used for classification tasks. The number of parameters includes the weights and biases.

## Summary of the Model

- **Total Parameters:** 23,514,179
- **Trainable Parameters:** 23,514,179
- **Non-trainable Parameters:** 0
- **Input Size (MB):** 0.57
- **Forward/Backward Pass Size (MB):** 286.55
- **Params Size (MB):** 89.70
- **Estimated Total Size (MB):** 376.82

## Explanation of the Summary

- **Total Parameters:** The total number of parameters in the model, including both trainable and non-trainable parameters.
- **Trainable Parameters:** The number of parameters that are updated during training.
- **Non-trainable Parameters:** The number of parameters that are not updated during training (e.g., fixed weights).
- **Input Size (MB):** The memory size of the input data.
- **Forward/Backward Pass Size (MB):** The memory size required for the forward and backward passes during training.
- **Params Size (MB):** The memory size of the model parameters.
- **Estimated Total Size (MB):** The estimated total memory size required for the model.

This detailed breakdown helps in understanding the architecture and complexity of the ResNet-like model, including the number of layers, their types, output shapes, and the number of parameters.

Transforming pretrained model: freezing layers and i# Set the model to evaluation mode
model.eval()ntroduce addtional ones

load parameters from previous model

In [ ]:
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pydicom
import cv2
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

# Set random seed for reproducibility
seed = 42  # You can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Set up the experiment in MLflow
experiment_name = "Resnet50_MRI_Classification_2"
mlflow.set_experiment(experiment_name)

# Specify the run ID and load the model from MLflow
run_id = "e2279f9e4abb43f9824a1ba402ce8168"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

# Load the model
m_1 = mlflow.pytorch.load_model(model_uri)

# Verify the model is loaded correctly
print("Loaded Model:")
print(m_1)

MRI transformation and loader settings

In [ ]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np  # Import numpy for setting the random seed

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),  # Uncomment if you want to use horizontal flipping
    # transforms.RandomRotation(10),     # Uncomment if you want to use rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading MRI images
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        # Ensure severity is in integer format
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']  # Use severity for the label

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')  # Normalize

        # Convert the image to RGB if it is grayscale
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Apply transformations including augmentation
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()  # Return label as tensor

    def __len__(self):
        return len(self.data)


Freezing layer and defining new one 

In [ ]:
# df_end = random_samples_test_check

In [ ]:
# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m_1.to(device)

# Freeze all layers of the loaded model
for param in m_1.parameters():
    param.requires_grad = False  # Freeze all parameters

# Adding BatchNorm1d helps in normalizing the output of each layer and can reduce the impact of 
# internal covariate shift, often leading to faster convergence.

# Add Batch Normalization:
m_1.fc = nn.Sequential(
    nn.Linear(m_1.fc.in_features, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(512, train_df['severity'].nunique())
)

just class one and two
reduce layeer geometically layers  512 divided by 4
1. If i will also fit the model for oter damages how to progess? Should it be done in separate steps using loops?

ugrade category 2 weights
U-net pretrained model

2. cases with medium and severity are undersampled in terms of study_id but equal in terms of images.

3. for 10000 pictures it took about 3 hours 



or 4. Add a Convolutional Layer: Adding a convolutional layer just before the fully connected layers might help /
extract more spatial features, especially if you have spatially informative data.

m_1.fc = nn.Sequential(
    nn.Conv2d(m_1.fc.in_features, 512, kernel_size=3, stride=1, padding=1),  # Convolutional layer
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.Dropout(0.5),
    nn.Linear(512, train_df['severity'].nunique())
)


Incorporating Attention Mechanisms:
Attention mechanisms (e.g., self-attention or squeeze-and-excitation layers) can help the model focus
on the most relevant parts of the image, which could be beneficial for medical image tasks.


o	Normal/Mild: Weight = 1
o	Moderate: Weight = 2
o	Severe: Weight = 4

# Move the modified model to the appropriate device (GPU or CPU) again, in case of changes
m_1.to(device)

# Set up training dataset and DataLoaders  
dataset = MRIDataset(data=train_df, transform=transform)



Setting train and validation datasets: claculating metrics CrossEntropyLoss 

In [ ]:
# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Define loss functions and optimizer
criterion_cel = nn.CrossEntropyLoss()
criterion_mse = nn.MSELoss()
optimizer = optim.Adam(m_1.parameters(), lr=0.0001)
num_epochs = 30

# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []
train_losses_mse = []
val_losses_mse = []

# Early stopping parameters
stop_threshold = 0.2  # Threshold for validation loss to diverge from training loss
diverge_count = 0
max_diverge_count = 3  # Number of epochs validation loss is allowed to diverge



introduce all mlflow log parameters see _1_1_1_model

setting mlflow: end running new model

In [ ]:
# Start MLflow run
with mlflow.start_run(nested=True):
    # Log parameters
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 4)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "Modified ResNet-50")
    mlflow.log_param("input_size", "224x224")
    mlflow.log_param("num_classes", train_df['severity'].nunique())

    # Training and validation loop
    for epoch in range(num_epochs):
        # Training phase
        m_1.train()
        running_loss_cel_train = 0.0
        running_loss_mse_train = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = m_1(images)
            
            # Cross Entropy Loss
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()

            # MSE Loss (if applicable)
            mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
            loss_mse = criterion_mse(outputs, mse_target)
            running_loss_mse_train += loss_mse.item()
            
            # Backward pass and optimization
            loss_cel.backward()
            optimizer.step()

        # Calculate average losses
        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)
        epoch_loss_mse_train = running_loss_mse_train / len(train_loader)
        train_losses_mse.append(epoch_loss_mse_train)

        # Log training losses to MLflow
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)
        mlflow.log_metric("train_loss_mse", epoch_loss_mse_train, step=epoch)

        # Validation phase
        m_1.eval()
        running_loss_cel_val = 0.0
        running_loss_mse_val = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = m_1(images)
                
                # Cross Entropy Loss for validation
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()
                
                # MSE Loss (if applicable)
                mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
                loss_mse = criterion_mse(outputs, mse_target)
                running_loss_mse_val += loss_mse.item()

        # Calculate validation losses
        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)
        epoch_loss_mse_val = running_loss_mse_val / len(val_loader)
        val_losses_mse.append(epoch_loss_mse_val)

        # Log validation losses to MLflow
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)
        mlflow.log_metric("val_loss_mse", epoch_loss_mse_val, step=epoch)

        # Early stopping check
        if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch+1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0  # Reset count if validation loss improves

        # Print epoch results
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

    print("Training complete!")

    # Log the model
    mlflow.pytorch.log_model(m_1, "modified_model")

    # passing to mlflow
   # Plot and log the loss curves as artifacts a
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")

    # Plot MSE Loss if applicable
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_mse, label='Train MSE Loss')
    plt.plot(val_losses_mse, label='Validation MSE Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation MSE Loss')
    plt.legend()
    plt.savefig("mse_loss.png")
    mlflow.log_artifact("mse_loss.png")



In [ ]:
# Plot and log the loss curves as artifacts
plt.figure(figsize=(10, 5))
plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Cross Entropy Loss')
plt.legend()
plt.savefig("cross_entropy_loss.png")
mlflow.log_artifact("cross_entropy_loss.png")
# Plot MSE Loss if applicable
plt.figure(figsize=(10, 5))
plt.plot(train_losses_mse, label='Train MSE Loss')
plt.plot(val_losses_mse, label='Validation MSE Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation MSE Loss')
plt.legend()
plt.savefig("mse_loss.png")
 


In [ ]:
# Count how many layers are frozen
frozen_layers_count = sum(1 for param in m_1.parameters() if not param.requires_grad)

# Get total number of layers (parameters)
total_layers_count = sum(1 for _ in m_1.parameters())

print(f"Frozen layers: {frozen_layers_count}/{total_layers_count} ({(frozen_layers_count / total_layers_count) * 100:.2f}%)")


In [ ]:
# Print the status of each layer
for name, param in m_1.named_parameters():
    if not param.requires_grad:
        print(f"Layer '{name}' is frozen.")
    else:
        print(f"Layer '{name}' is trainable.")

In [ ]:
test_data = test_df

In [ ]:
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



In [ ]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


In [ ]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
test_df['Probability_Class_2'] = probabilities_array[:, 2]

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
test_df['Probability_Class_2'] = test_df['Probability_Class_2'].round(4)
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

In [ ]:
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df.groupby('severity')[['Probability_Class_0', 'Probability_Class_1', 'Probability_Class_2']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


In [ ]:
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Ensure true_labels and predicted_labels have the same length
min_length = min(len(true_labels), len(predicted_labels))
true_labels = true_labels[:min_length]
predicted_labels = predicted_labels[:min_length]

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset, random_split
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms

# Set random seed for reproducibility
seed = 42  # You can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),  # Uncomment if you want to use horizontal flipping
    # transforms.RandomRotation(10),     # Uncomment if you want to use rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Ensure your dataset uses this transform
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        # Ensure severity is in integer format
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']  # Use severity for the label

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')  # Normalize

        # Convert the image to RGB if it is grayscale
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Apply transformations including augmentation
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()  # Return label as tensor

    def __len__(self):
        return len(self.data)

# Set up the experiment in MLflow
experiment_name = "Resnet50_MRI_Classification"
mlflow.set_experiment(experiment_name)

# Specify the run ID and load the model from MLflow
run_id = "f390913c59d642329c86d0f52b943062"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

# Load the model
m_1 = mlflow.pytorch.load_model(model_uri)

# Verify the model is loaded correctly
print("Loaded Model:")
print(m_1)

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m_1.to(device)

# Create the dataset with transformations
dataset = MRIDataset(data=random_samples_test_check, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Define loss functions and optimizer
criterion_cel = nn.CrossEntropyLoss()
criterion_mse = nn.MSELoss()
optimizer = optim.Adam(m_1.parameters(), lr=0.0001)
num_epochs = 30

# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []
train_losses_mse = []
val_losses_mse = []

# Early stopping parameters
stop_threshold = 0.2  # Threshold for validation loss to diverge from training loss
diverge_count = 0
max_diverge_count = 3  # Number of epochs validation loss is allowed to diverge

# Start MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 4)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "Modified ResNet-50")
    mlflow.log_param("input_size", "224x224")
    mlflow.log_param("num_classes", df_end['severity'].nunique())

    # Training and validation loop
    for epoch in range(num_epochs):
        # Training phase
        m_1.train()
        running_loss_cel_train = 0.0
        running_loss_mse_train = 0.0
        
        for images, labels in train_loader:
            if images is None or images.size(0) == 0:
                continue
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = m_1(images)
            
            # Cross Entropy Loss
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()

            # MSE Loss (if applicable)
            mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
            loss_mse = criterion_mse(outputs, mse_target)
            running_loss_mse_train += loss_mse.item()
            
            # Backward pass and optimization
            loss_cel.backward()
            optimizer.step()

        # Calculate average losses
        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)
        epoch_loss_mse_train = running_loss_mse_train / len(train_loader)
        train_losses_mse.append(epoch_loss_mse_train)

        # Log training losses to MLflow
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)
        mlflow.log_metric("train_loss_mse", epoch_loss_mse_train, step=epoch)

        # Validation phase
        m_1.eval()
        running_loss_cel_val = 0.0
        running_loss_mse_val = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = m_1(images)
                
                # Cross Entropy Loss for validation
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()
                
                # MSE Loss (if applicable)
                mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
                loss_mse = criterion_mse(outputs, mse_target)
                running_loss_mse_val += loss_mse.item()

        # Calculate validation losses
        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)
        epoch_loss_mse_val = running_loss_mse_val / len(val_loader)
        val_losses_mse.append(epoch_loss_mse_val)

        # Log validation losses to MLflow
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)
        mlflow.log_metric("val_loss_mse", epoch_loss_mse_val, step=epoch)

        # Early stopping check
        if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch+1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0  # Reset count if validation loss improves

        # Print epoch results
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

    print("Training complete!")

    # Log the model
    mlflow.pytorch.log_model(m_1, "modified_model")

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")

    # Plot MSE Loss if applicable
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_mse, label='Train MSE Loss')
    plt.plot(val_losses_mse, label='Validation MSE Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation MSE Loss')
    plt.legend()
    plt.savefig("mse_loss.png")
    mlflow.log_artifact("mse_loss.png")


In [ ]:
s

In [ ]:
s

In [ ]:
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Set up the experiment name and load the existing model
experiment_name = "Resnet50_MRI_Classification"
mlflow.set_experiment(experiment_name)

# Specify the run ID of the previously logged model
run_id = "f390913c59d642329c86d0f52b943062"  # Replace with your actual run ID

# Create the model URI
model_uri = f"runs:/{run_id}/model"

# Load the original model from MLflow
m_1 = mlflow.pytorch.load_model(model_uri)

# Verify that the original model is loaded correctly
print("Original Model Loaded:")
print(m_1)

# Freeze all layers of the loaded model
for param in m_1.parameters():
    param.requires_grad = False  # Freeze all parameters

# Modify the fully connected (fc) layer to match new requirements
m_1.fc = nn.Sequential(
    nn.Linear(m_1.fc.in_features, 512),  # Intermediate fully connected layer
    nn.ReLU(),                           # Activation function
    nn.Dropout(0.5),                     # Optional regularization with dropout
    nn.Linear(512, m_1.fc.out_features)  # Final layer for classification
)

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m_1.to(device)

# Print the modified model architecture to verify changes
print("Modified Model Architecture:")
print(m_1)

# Log the new modified model in MLflow
with mlflow.start_run():
    mlflow.log_param("model_architecture", "Modified ResNet-50 based on original model")
    mlflow.pytorch.log_model(m_1, "modified_model")

print("New modified model logged successfully.")


In [ ]:
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Set random seed for reproducibility
seed = 42  # You can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Set up the experiment in MLflow
experiment_name = "Resnet50_MRI_Classification"
mlflow.set_experiment(experiment_name)

# Specify the run ID and load the model from MLflow
run_id = "f390913c59d642329c86d0f52b943062"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

# Load the model
m_1 = mlflow.pytorch.load_model(model_uri)

# Verify the model is loaded correctly
print("Loaded Model:")
print(m_1)

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
m_1.to(device)

# Set up dataset and DataLoaders (assume MRIDataset and transform are defined)
dataset = MRIDataset(data=random_samples_test_check, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Define loss functions and optimizer
criterion_cel = nn.CrossEntropyLoss()
criterion_mse = nn.MSELoss()
optimizer = optim.Adam(m_1.parameters(), lr=0.0001)
num_epochs = 30

# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []
train_losses_mse = []
val_losses_mse = []

# Early stopping parameters
stop_threshold = 0.2  # Threshold for validation loss to diverge from training loss
diverge_count = 0
max_diverge_count = 3  # Number of epochs validation loss is allowed to diverge

# Start MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 4)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "Modified ResNet-50")
    mlflow.log_param("input_size", "224x224")
    mlflow.log_param("num_classes", df_end['severity'].nunique())

    # Training and validation loop
    for epoch in range(num_epochs):
        # Training phase
        m_1.train()
        running_loss_cel_train = 0.0
        running_loss_mse_train = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = m_1(images)
            
            # Cross Entropy Loss
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()

            # MSE Loss (if applicable)
            mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
            loss_mse = criterion_mse(outputs, mse_target)
            running_loss_mse_train += loss_mse.item()
            
            # Backward pass and optimization
            loss_cel.backward()
            optimizer.step()

        # Calculate average losses
        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)
        epoch_loss_mse_train = running_loss_mse_train / len(train_loader)
        train_losses_mse.append(epoch_loss_mse_train)

        # Log training losses to MLflow
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)
        mlflow.log_metric("train_loss_mse", epoch_loss_mse_train, step=epoch)

        # Validation phase
        m_1.eval()
        running_loss_cel_val = 0.0
        running_loss_mse_val = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = m_1(images)
                
                # Cross Entropy Loss for validation
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()
                
                # MSE Loss (if applicable)
                mse_target = labels.float().unsqueeze(1).expand_as(outputs)  # Ensure shape matches
                loss_mse = criterion_mse(outputs, mse_target)
                running_loss_mse_val += loss_mse.item()

        # Calculate validation losses
        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)
        epoch_loss_mse_val = running_loss_mse_val / len(val_loader)
        val_losses_mse.append(epoch_loss_mse_val)

        # Log validation losses to MLflow
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)
        mlflow.log_metric("val_loss_mse", epoch_loss_mse_val, step=epoch)

        # Early stopping check
        if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch+1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0  # Reset count if validation loss improves

        # Print epoch results
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

    print("Training complete!")

    # Log the model
    mlflow.pytorch.log_model(m_1, "modified_model")

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")

    # Plot MSE Loss if applicable
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_mse, label='Train MSE Loss')
    plt.plot(val_losses_mse, label='Validation MSE Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation MSE Loss')
    plt.legend()
    plt.savefig("mse_loss.png")
    mlflow.log_artifact("mse_loss.png")


In [ ]:
# Display the model summary
summary(model, input_size=(3, 224, 224))

In [ ]:
import mlflow
import mlflow.pytorch
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import pydicom
import cv2
import pandas as pd

# Assuming random_samples_test_check is your test dataset DataFrame
test_data = random_samples_test_check

# Define your dataset class if not defined yet
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']
        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')
        if len(image.shape) == 2:  # Convert grayscale to RGB
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)
        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

# Transform for test pk
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_data = random_samples_test_check
# Create the test dataset and DataLoader
test_dataset = MRIDataset(data=random_samples_test_check, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Load the model
experiment_name = "Resnet50_MRI_Classification"
mlflow.set_experiment(experiment_name)

run_id = "f390913c59d642329c86d0f52b943062"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

model = mlflow.pytorch.load_model(model_uri)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move model to the appropriate device
model.eval()  # Set the model to evaluation mode

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        labels = labels.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Ensure labels are also on the correct device
        
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)

        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())

# Convert results and probabilities to DataFrame for easier handling
results_df = pd.DataFrame(results, columns=['Predicted_Class'])
probabilities_df = pd.DataFrame(np.vstack(probabilities_list), columns=[f'Class_{i}' for i in range(probabilities.shape[1])])

# Combine predictions and probabilities
final_results = pd.concat([results_df, probabilities_df], axis=1)

# Save results to a CSV file
final_results.to_csv("test_predictions.csv", index=False)

# print("Predictions and probabilities have been saved to test_predictions.csv.")


In [ ]:
test_data = random_samples_test_check

test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



In [ ]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


In [ ]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Calculate the average probability for each class
average_probabilities = np.mean(probabilities_array, axis=0)

# Print the average probabilities
for i, avg_prob in enumerate(average_probabilities):
    print(f"Average probability for class {i}: {avg_prob:.4f}")